In [2]:
import pdfplumber
import pandas as pd

# Chemin du PDF
pdf_path = r"C:\Users\basti\Desktop\Bank_analysis\data\Avril2025.pdf"

# Liste pour stocker les lignes
rows = []

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        tables = page.extract_tables()
        for table in tables:
            for row in table:
                if row and any(cell and cell.strip() for cell in row):
                    rows.append(row)

# Vérifie si des données ont été extraites
if not rows:
    print("Aucune donnée extraite. Vérifie que ton PDF contient bien du texte (pas une image).")
else:
    # Affiche un aperçu de la première ligne
    print("Première ligne (en-têtes supposés) :", rows[0])

    # Crée le DataFrame
    df = pd.DataFrame(rows[1:], columns=rows[0])

    # Optionnel : nettoyage de colonnes (à adapter selon ta structure)
    # Exemple : conversion des montants
    try:
        df["Montant"] = df["Montant"].str.replace(",", ".").str.replace("€", "").astype(float)
    except Exception:
        pass  # On ignore si la colonne n'existe pas ou si la conversion échoue

    # Export CSV
    output_csv = pdf_path.replace(".pdf", ".csv")
    df.to_csv(output_csv, index=False, encoding="utf-8-sig")
    print(f"✅ CSV exporté : {output_csv}")


Première ligne (en-têtes supposés) : ['Date\nopé.', 'Date\nvaleur', 'Libellé des opérations', 'Débit', 'Crédit', 'þ']
✅ CSV exporté : C:\Users\basti\Desktop\Bank_analysis\data\Avril2025.csv


In [4]:
import pandas as pd

# Charger le fichier CSV nettoyé depuis pdfplumber
df = pd.read_csv("C:/Users/basti/Desktop/Bank_analysis/data/Avril2025.csv", encoding="utf-8-sig")

# Afficher les premières lignes
df.head()


,Date\nopé.,Date\nvaleur,Libellé des opérations,Débit,Crédit,þ
0,NaN,NaN,Ancien solde créditeur au 05.03.2025,NaN,"1 560,95",NaN
1,06.03,06.03,Carte X2648 Mix Markt 131 Aalen 05/03,"4,39",NaN,¨
2,07.03,07.03,Virement De Deffradas Bruno,NaN,"800,00",¨
3,07.03,07.03,Carte X2648 Rewe Kurz ohg Aalen 06/03,"16,11",NaN,¨
4,10.03,10.03,Carte X2648 Action 3227 Aalen-un 08/03,"2,99",NaN,¨


In [5]:
# Nettoyage des noms de colonnes
df.columns = (
    df.columns
    .str.replace("\n", " ")  # remplacer les sauts de ligne
    .str.strip()
    .str.lower()
)

# Afficher pour vérifier
print(df.columns)


Index(['date opé.', 'date valeur', 'libellé des opérations', 'débit', 'crédit',
       'þ'],
      dtype='object')


In [6]:
# Supprimer la colonne 'þ' ou toute colonne sans nom utile
colonnes_utiles = ['date opé.', 'date valeur', 'libellé des opérations', 'débit', 'crédit']
df = df[[col for col in df.columns if col in colonnes_utiles]]


In [7]:
df = df.rename(columns={
    'date valeur': 'date',
    'libellé des opérations': 'description'
})


In [8]:
df.head()

,date opé.,date,description,débit,crédit
0,NaN,NaN,Ancien solde créditeur au 05.03.2025,NaN,"1 560,95"
1,06.03,06.03,Carte X2648 Mix Markt 131 Aalen 05/03,"4,39",NaN
2,07.03,07.03,Virement De Deffradas Bruno,NaN,"800,00"
3,07.03,07.03,Carte X2648 Rewe Kurz ohg Aalen 06/03,"16,11",NaN
4,10.03,10.03,Carte X2648 Action 3227 Aalen-un 08/03,"2,99",NaN


In [9]:
# Nettoyage des montants dans débit et crédit
for col in ["débit", "crédit"]:
    if col in df.columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.replace(",", ".")
            .str.replace("€", "")
            .str.replace(" ", "")
            .str.strip()
        )
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Affichage pour vérification
df[["date", "description", "débit", "crédit"]].head()


,date,description,débit,crédit
0,NaN,Ancien solde créditeur au 05.03.2025,NaN,1560.95
1,06.03,Carte X2648 Mix Markt 131 Aalen 05/03,4.39,NaN
2,07.03,Virement De Deffradas Bruno,NaN,800.00
3,07.03,Carte X2648 Rewe Kurz ohg Aalen 06/03,16.11,NaN
4,10.03,Carte X2648 Action 3227 Aalen-un 08/03,2.99,NaN


In [10]:
# Calcul du montant net
df["montant"] = df["crédit"].fillna(0) - df["débit"].fillna(0)

# Aperçu pour contrôle
df[["date", "description", "débit", "crédit", "montant"]].head()


,date,description,débit,crédit,montant
0,NaN,Ancien solde créditeur au 05.03.2025,NaN,1560.95,1560.95
1,06.03,Carte X2648 Mix Markt 131 Aalen 05/03,4.39,NaN,-4.39
2,07.03,Virement De Deffradas Bruno,NaN,800.00,800.00
3,07.03,Carte X2648 Rewe Kurz ohg Aalen 06/03,16.11,NaN,-16.11
4,10.03,Carte X2648 Action 3227 Aalen-un 08/03,2.99,NaN,-2.99


In [13]:
import re

# Fonction pour extraire une date de type 05/03 ou 06.03 dans le texte
def extraire_date_depuis_description(desc):
    match = re.search(r"(\d{2})[/.](\d{2})", str(desc))
    if match:
        jour, mois = match.groups()
        return pd.to_datetime(f"{jour}.{mois}.2025", format="%d.%m.%Y", errors="coerce")
    return pd.NaT

# Appliquer la fonction sur la colonne description
df["date"] = df["description"].apply(extraire_date_depuis_description)

# Vérification
df[["date", "description", "montant"]].head()


,date,description,montant
0,2025-03-05,Ancien solde créditeur au 05.03.2025,1560.95
1,2025-03-05,Carte X2648 Mix Markt 131 Aalen 05/03,-4.39
2,NaT,Virement De Deffradas Bruno,800.00
3,2025-03-06,Carte X2648 Rewe Kurz ohg Aalen 06/03,-16.11
4,2025-03-08,Carte X2648 Action 3227 Aalen-un 08/03,-2.99


In [14]:
# Afficher les mots ou débuts de libellés les plus fréquents
df["description"].str.lower().str.split().str[0].value_counts().head(20)


description
carte        65
virement      7
ret           4
libellé       2
prlv          2
ancien        1
annul.        1
deffradas     1
pel           1
rundfunk      1
bequem:       1
core          1
cotis         1
total         1
nouveau       1
Name: count, dtype: int64

In [15]:
# Export du fichier final propre
df_final = df[["date", "description", "montant"]]
df_final.to_csv("C:/Users/basti/Desktop/Bank_analysis/data/Avril2025_final.csv", index=False, encoding="utf-8-sig")

print("✅ Fichier exporté : Avril2025_final.csv")


✅ Fichier exporté : Avril2025_final.csv
